# Eat Safe, Love

## Notebook Set Up

In [8]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

import pandas as pd

In [9]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [10]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [11]:
# review the collections in our database
print(db.list_collection_names())

ServerSelectionTimeoutError: localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it, Timeout: 30s, Topology Description: <TopologyDescription id: 64ee9a53c3435e18c5fcebde, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it')>]>

In [ ]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [ ]:
#  view the first document
establishments.find_one()

In [ ]:
# Find the establishments with a hygiene score of 20
query_1 = {'scores.Hygiene': 20}

query_results = establishments.find(query_1)
# Use count_documents to display the number of documents in the result
results_1 = establishments.count_documents(query_1)
print(f"There are {results_1} establishments with a 'Hygiene' score of 20.")

# Display the first document in the results using pprint
first_document = establishments.find_one(query_1)
pprint(first_document)



In [ ]:
# Convert the result to a Pandas DataFrame
hygiene_df = pd.DataFrame(establishments.find(query_1))

# Display the number of rows in the DataFrame
count_of_hygiene_df = len(hygiene_df)
print(f"There are {count_of_hygiene_df} rows within this Hygiene Dataframe")

# Display the first 10 rows of the DataFrame
hygiene_df.head(10)

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query_2 = {'LocalAuthorityName': {'$regex': "London"},"RatingValue":{'$gte': 4}}

# Use count_documents to display the number of documents in the result
results_2 = establishments.count_documents(query_2)

print(f"There are {results_2} establishments with a 'RatingValue' score of 4 or above.")


# Display the first document in the results using pprint
first_document = establishments.find_one(query_2)
pprint(first_document)



In [ ]:
# Convert the result to a Pandas DataFrame
rating_df = pd.DataFrame(establishments.find(query_2))

# Display the number of rows in the DataFrame
count_of_rating_df = len(rating_df)
print(f"There are {count_of_rating_df} rows within this Hygiene Dataframe")

# Display the first 10 rows of the DataFrame
rating_df.head(10)

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# get latitude and longitude for Penang Flavours
query = {'BusinessName': 'Penang Flavours'}
fields = {'geocode': 1}

pprint(establishments.find_one(query, fields))

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score
degree_search = 0.01
latitude = 51.49014200
longitude = 0.08384000

query_3 = {
    "RatingValue": 5,
    'geocode.latitude': {
        '$lte': latitude + degree_search,
        '$gte': latitude - degree_search
    },
    'geocode.longitude': {
        '$lte': longitude + degree_search,
        '$gte': longitude - degree_search
    }
}

#Sort by hygiene score
sort = [('scores.Hygiene', 1)]

# Print the results
results_Q3 = list(establishments.find(query_3).sort(sort))

pprint(results_Q3)


In [ ]:
# Convert result to Pandas DataFrame
top5_penang_flavours_df = pd.DataFrame(results_Q3)
top5_penang_flavours_df


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:


# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

# Create match query
match_query = {"$match": {'scores.Hygiene': 0}}

# Grouped by "Local Authority"
group_query = {'$group': {'_id': '$LocalAuthorityName', 'count': {'$sum': 1}}}

# Then sort in decending order
sort_values = {'$sort': {'count': -1}}

# Create 'pipeline' function
pipeline = [match_query, group_query, sort_values]

# Print the number of documents in the result
results_Q4 = list(establishments.aggregate(pipeline))
print(f"Number of documents in result: {len(results_Q4)}")

# Print the first 10 results
pprint(results_Q4[0:10])
    



In [ ]:

# Convert the result to a Pandas DataFrame
Q4_df = pd.DataFrame(results_Q4)

# Display the number of rows in the DataFrame
print(f"There are {len(Q4_df)} rows within this Local Authority count DataFrame")

# Display the first 10 rows of the DataFrame
Q4_df.head(10)

